In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib Tk

## Day 25

In [1]:
subject_number = 7
card_public_key = 7573546
door_public_key = 17786549
div_num = 20201227

In [2]:
def transform(subject_number, loop_size=None):
    value = 1
    for i in range(loop_size):
        value *= subject_number
        value = value % div_num
    return value

In [3]:
def transform_generator(subject_number):
    value = 1
    while True:
        value *= subject_number
        value = value % div_num
        yield value

In [4]:
def decode_loop_size(subject_number, public_key):
    found = False
    for loop_size, pk in enumerate(transform_generator(subject_number)):
        if pk == public_key:
            break
    return loop_size + 1

In [5]:
card_loop_size = decode_loop_size(subject_number, card_public_key)
card_loop_size

10985209

In [6]:
door_loop_size = decode_loop_size(subject_number, door_public_key)
door_loop_size

925199

In [7]:
transform(card_public_key, door_loop_size)

7032853

In [8]:
transform(door_public_key, card_loop_size)

7032853

## Day 24


In [9]:
class Sequence:
    
    def __init__(self, sequence):
        self.sequence = list(sequence)
        self.valid_directions = ["e", "se", "sw", "w", "nw", "ne"]
        
    def __iter__(self):
        return self
        
    def __next__(self):
        try:
            direction = self.sequence.pop(0)
            if direction not in self.valid_directions:
                direction += self.sequence.pop(0)
        except IndexError:
            raise StopIteration
        return direction
        

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib osx

class HexGrid:
    
    def __init__(self):
        self.reference_tile = [0, 0]
        self.tile_color = {}
        
    def move(self, tile, direction):
        if direction == "e":
            tile[0] += 0
            tile[1] += 1
        elif direction == "se":
            tile[0] -= 0.5
            tile[1] += 0.5
        elif direction == "sw":
            tile[0] -= 0.5
            tile[1] -= 0.5
        elif direction == "w":
            tile[0] += 0
            tile[1] -= 1
        elif direction == "nw":
            tile[0] += 0.5
            tile[1] -= 0.5
        elif direction == "ne":
            tile[0] += 0.5
            tile[1] += 0.5
        if tuple(tile) not in self.tile_color:
            self.tile_color[tuple(tile)] = "white"
        return tile
         
    def locate_tile(self, sequence):
        tile = self.reference_tile.copy()
        for s in sequence:
            tile = self.move(tile, s)
        return tile
    
    def flip_tile(self, sequence=None, tile=None):
        if tile is None:
            tile = self.locate_tile(sequence)
        color = "white" if tuple(tile) not in self.tile_color else self.tile_color[tuple(tile)]
        new_color = ["black", "white"][int(color == "black")]
        self.tile_color[tuple(tile)] = new_color
        
    def fill_adjacent_colors(self):
        for tile, color in list(self.tile_color.items()):
            if color == "white":
                continue
            for i, j in [(0, -1), (0.5, -0.5), (1, 0), (0.5, 0.5), (0, 1), (-0.5, 0.5), (-1, 0), (-0.5, -0.5)]:
                adj_tile = (tile[0] + i, tile[1] + j)
                if adj_tile not in self.tile_color:
                    self.tile_color[adj_tile] = "white"
        
    def get_adjacent_colors(self, tile):
        n_adjacent = 6
        adjacent_colors = []
        for i, j in [(0, -1), (0.5, -0.5), (0.5, 0.5), (0, 1), (-0.5, 0.5), (-0.5, -0.5)]:
            adj_tile = (tile[0] + i, tile[1] + j)
            assert len(adj_tile) <= n_adjacent
            try:
                adjacent_colors.append(self.tile_color[adj_tile])
            except KeyError:
                pass
        return adjacent_colors
        
    def apply_white_rule(self, adj_colors):
        n_black = len([c for c in adj_colors if c == "black"])
        return n_black == 2            
    
    def apply_black_rule(self, adj_colors):
        n_white = len([c for c in adj_colors if c == "black"])
        return n_white == 0 or n_white > 2
    
    def make_art(self):
        tiles_to_flip = []
        for tile, color in self.tile_color.items():
            adj_colors = self.get_adjacent_colors(tile)
            if color == "white":
                to_flip = self.apply_white_rule(adj_colors)
            else:
                to_flip = self.apply_black_rule(adj_colors)
            if to_flip:
                tiles_to_flip.append(tile)
        for t in tiles_to_flip:
            self.flip_tile(tile=t)
        
    def n_black_tiles(self):
        return len([t for t in self.tile_color.values() if t == "black"])
    
    def plot_tiles(self, ax=None):
        update = False
        if ax is None:
            fig, ax = plt.subplots(1, 1, figsize=(10, 10))
            ax.set_facecolor('grey')
        else:
            ax.clear()
            update = True
        data = []
        for tile, color in self.tile_color.items():
            data.append([tile[1], tile[0], 0 if color == "white" else 1]) 
        df = pd.DataFrame(data=data, columns=["x", "y", "tile_color"])
        ax.scatter(data=df, x="x", y="y", c="tile_color", cmap="binary")
        sns.despine(bottom=True, left=True)
        plt.show()
        return ax

In [11]:
floor = HexGrid()
with open("/Users/mikes/Downloads/input.txt", 'r') as fin:
    for line in fin.readlines():
        sequence = Sequence(line)
        floor.flip_tile(sequence)
floor.n_black_tiles()

244

In [15]:
floor.fill_adjacent_colors()
floor.plot_tiles()
for i in range(100):
    floor.make_art()
    floor.fill_adjacent_colors()
floor.plot_tiles()
floor.n_black_tiles()

KeyboardInterrupt: 